<a href="https://colab.research.google.com/github/ekvirika/WalmartRecruiting/blob/main/notebooks/model_experiment_lightgbm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:

# Install required packages
!pip install -q wandb torch torchvision pandas numpy matplotlib seaborn scikit-learn mlflow pytorch_lightning pytorch_forecasting

# Set up Kaggle API
!pip install -q kaggle  dagshub

In [17]:

# Upload your kaggle.json to Colab and run:
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/ColabNotebooks/kaggle_API_credentials/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json


In [18]:

# Download the dataset
!kaggle competitions download -c walmart-recruiting-store-sales-forecasting
!unzip -q walmart-recruiting-store-sales-forecasting.zip

walmart-recruiting-store-sales-forecasting.zip: Skipping, found more recently modified local copy (use --force to force download)
replace features.csv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace sampleSubmission.csv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace stores.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace test.csv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace train.csv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: y


In [19]:

!unzip -q train.csv.zip
!unzip -q stores.csv.zip
!unzip -q test.csv.zip
!unzip -q features.csv.zip


replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
unzip:  cannot find or open stores.csv.zip, stores.csv.zip.zip or stores.csv.zip.ZIP.
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace features.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y


In [20]:
!pip install dagshub mlflow --quiet
! pip install optuna

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime, timedelta

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.base import BaseEstimator, TransformerMixin

import lightgbm as lgb
import mlflow
import mlflow.lightgbm
from mlflow.tracking import MlflowClient

warnings.filterwarnings('ignore')


In [22]:
import mlflow
from dagshub import dagshub_logger
import os

# Set tracking URI manually
mlflow.set_tracking_uri("https://dagshub.com/ekvirika/WalmartRecruiting.mlflow")

# Use your DagsHub credentials
os.environ["MLFLOW_TRACKING_USERNAME"] = "ekvirika"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "0adb1004ddd4221395353efea2d8ead625e26197"

# Optional: set registry if you're using model registry
mlflow.set_registry_uri("https://dagshub.com/ekvirika/WalmartRecruiting.mlflow")

In [23]:
# import mlflow.sklearn
# import mlflow.lightgbm
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.impute import SimpleImputer
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from lightgbm import LGBMRegressor
# from sklearn.metrics import mean_absolute_error
# from sklearn.model_selection import RandomizedSearchCV
# import matplotlib.pyplot as plt
# import joblib




# # ----------------------
# # Evaluation Metric (WMAE)
# # ----------------------
# def weighted_mae(y_true, y_pred, is_holiday):
#     weights = np.where(is_holiday, 5, 1)
#     return np.sum(weights * np.abs(y_true - y_pred)) / np.sum(weights)

# # ----------------------
# # Data Loader
# # ----------------------
# def load_data():
#     with mlflow.start_run(run_name="LGBM_Data_Loading", nested=True):
#         train_df = pd.read_csv('train.csv')
#         test_df = pd.read_csv('test.csv')
#         stores_df = pd.read_csv('stores.csv')
#         features_df = pd.read_csv('features.csv')

#         df = train_df.merge(features_df, on=['Store', 'Date'], how='inner') \
#                      .merge(stores_df, on='Store', how='inner')

#         mlflow.log_params({
#             "train_shape": train_df.shape,
#             "test_shape": test_df.shape,
#             "missing_values_train": train_df.isnull().sum().sum(),
#             "missing_values_test": test_df.isnull().sum().sum()
#         })

#         return df

# # ----------------------
# # Preprocessing
# # ----------------------
# def preprocess(df):
#     with mlflow.start_run(run_name="LGBM_Cleaning", nested=True):


#       df = df.copy()

#       # Fix column naming issues
#       df["IsHoliday"] = df.pop("IsHoliday_x") if "IsHoliday_x" in df else df["IsHoliday"]
#       df.drop(columns=["IsHoliday_y"], errors='ignore', inplace=True)

#       # Filter and sort
#       df = df[df["Weekly_Sales"] > 0]
#       df["Date"] = pd.to_datetime(df["Date"])
#       df = df.sort_values(by="Date")

#       # Remove outliers
#       q_low = df["Weekly_Sales"].quantile(0.01)
#       q_high = df["Weekly_Sales"].quantile(0.99)
#       df = df[(df["Weekly_Sales"] >= q_low) & (df["Weekly_Sales"] <= q_high)]

#       # Lag features
#       for lag in [1, 2, 4, 52]:
#           df[f"lag_{lag}"] = df.groupby("Store")["Weekly_Sales"].shift(lag)

#       # Rolling statistics
#       df["rolling_mean_4"] = df.groupby("Store")["Weekly_Sales"].shift(1).rolling(window=4).mean()
#       df["rolling_std_4"] = df.groupby("Store")["Weekly_Sales"].shift(1).rolling(window=4).std()

#       # Fill missing MarkDowns
#       markdown_cols = [col for col in df.columns if "MarkDown" in col]
#       df[markdown_cols] = df[markdown_cols].fillna(0)

#       # Date features
#       df["Year"] = df["Date"].dt.year
#       df["Month"] = df["Date"].dt.month
#       df["Week"] = df["Date"].dt.isocalendar().week
#       df["DayOfWeek"] = df["Date"].dt.dayofweek
#       df["IsMonthStart"] = df["Date"].dt.is_month_start.astype(int)
#       df["IsMonthEnd"] = df["Date"].dt.is_month_end.astype(int)
#       df["Quarter"] = df["Date"].dt.quarter
#       df.drop(columns=["Date"], inplace=True)

#       mlflow.log_params({"droppped_cols": "IsHoliday_x, IsHoliday_y, Date", "fill_markdown_NaNs": "0"})

#       # Drop rows with missing lag/rolling values
#       df = df.dropna()

#       return df

# # ----------------------
# # Dynamic Pipeline Builder
# # ----------------------
# def build_pipeline(X, model=None):
#     with mlflow.start_run(run_name="LGBM_Feature_Engineering", nested=True):

#       numeric_cols = X.select_dtypes(include=np.number).columns.tolist()
#       categorical_cols = X.select_dtypes(include=["object", "category", "bool"]).columns.tolist()

#       preprocessor = ColumnTransformer(transformers=[
#           ("num", SimpleImputer(strategy='mean'), numeric_cols),
#           ("cat", OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_cols)
#       ])

#       if model is None:
#           model = LGBMRegressor(
#               n_estimators=100,
#               max_depth=6,
#               learning_rate=0.1,
#               random_state=42,
#               n_jobs=-1
#           )

#       pipeline = Pipeline([
#           ("preprocessor", preprocessor),
#           ("model", model)
#       ])

#       mlflow.log_params({"num_cols": numeric_cols, "cat_cols": categorical_cols,
#                         "simple_imputer": "mean", "cat_encoder": "one_hot", "pipeline": pipeline})

#       return pipeline

# # ----------------------
# # Runner
# # ----------------------
# def run():
#     # with mlflow.start_run(run_name="XGBoost_Training"):

#       df = load_data()
#       df = preprocess(df)

#       # Define features and target
#       X = df.drop(columns=["Weekly_Sales"])
#       y = df["Weekly_Sales"]

#       # Split
#       X_train, X_test, y_train, y_test = train_test_split(
#           X, y, test_size=0.2, random_state=42
#       )

#       # Store IsHoliday for WMAE
#       is_holiday_test = X_test["IsHoliday"].astype(bool).values if "IsHoliday" in X_test.columns else np.zeros_like(y_test)

#       # Train
#       pipeline = build_pipeline(X_train)

#       # Hyperparameter search space
#       param_dist = {
#           "model__n_estimators": [100, 200, 300, 500, 800, 1000],
#           "model__max_depth": [3, 6, 9, -1],  # -1 means no limit
#           "model__learning_rate": [0.01, 0.1],
#           "model__subsample": [0.6, 0.8, 1.0],  # still valid
#           "model__colsample_bytree": [0.6, 0.8, 1.0],  # also valid
#           "model__num_leaves": [31, 63, 127, 255, 511]
#       }


#       search = RandomizedSearchCV(
#           pipeline,
#           param_distributions=param_dist,
#           n_iter=20,
#           cv=3,
#           scoring="neg_mean_absolute_error",  # you could also define custom scorer using WMAE if needed
#           verbose=2,
#           n_jobs=-1,
#           random_state=42
#       )

#       search.fit(X_train, y_train)

#       best_model = search.best_estimator_
#       y_pred = best_model.predict(X_test)

#       # WMAE
#       wmae_score = weighted_mae(y_test.values, y_pred, is_holiday_test)
#       print(f"✅ WMAE: {wmae_score:.2f}")

#       mlflow.log_params(search.best_params_)
#       mlflow.log_metric("WMAE", wmae_score)
#       # mlflow.sklearn.log_model(best_model.named_steps["model"], artifact_path="lgbm_model")
#       joblib.dump(best_model.named_steps["model"], "lgbm_model.pkl")
#       mlflow.log_artifact("lgbm_model.pkl")

#       joblib.dump(best_model, "lgbm_pipeline.pkl")
#       mlflow.log_artifact("lgbm_pipeline.pkl")


#       # Save feature importance plot as artifact
#       fig, ax = plt.subplots()
#       lgb_model = best_model.named_steps["model"]
#       importances = lgb_model.feature_importances_
#       ax.bar(range(len(importances)), importances)
#       plt.savefig("feature_importance.png")
#       mlflow.log_artifact("feature_importance.png")


#       return best_model

# # ----------------------
# # Entry Point
# # ----------------------
# if __name__ == "__main__":
#     try:
#         with mlflow.start_run(run_name="LGBM_Training"):
#             model = run()
#         print("✅ Pipeline executed successfully!")
#     except Exception as e:
#         print(f"❌ Pipeline failed: {e}")


In [24]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
stores_df = pd.read_csv('stores.csv')
features_df = pd.read_csv('features.csv')

In [25]:
import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn
import mlflow.lightgbm
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import joblib

# ----------------------
# Evaluation Metric (WMAE)
# ----------------------
def weighted_mae(y_true, y_pred, is_holiday):
    weights = np.where(is_holiday, 5, 1)
    return np.sum(weights * np.abs(y_true - y_pred)) / np.sum(weights)

# ----------------------
# Custom Transformers
# ----------------------
class DataMerger(BaseEstimator, TransformerMixin):
    def __init__(self, features_df, stores_df):
        self.features_df = features_df
        self.stores_df = stores_df

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X = X.merge(self.features_df, on=['Store', 'Date'], how='inner')
        X = X.merge(self.stores_df, on='Store', how='inner')
        return X

class DataCleaner(BaseEstimator, TransformerMixin):
    def __init__(self, is_train=True):
        self.q_low = None
        self.q_high = None
        self.is_train = is_train

    def fit(self, X, y=None):
        if self.is_train and 'Weekly_Sales' in X.columns:
            self.q_low = X["Weekly_Sales"].quantile(0.01)
            self.q_high = X["Weekly_Sales"].quantile(0.99)
        return self

    def transform(self, X):
        X = X.copy()

        if "IsHoliday_x" in X.columns:
            X["IsHoliday"] = X.pop("IsHoliday_x")
        X.drop(columns=["IsHoliday_y"], errors='ignore', inplace=True)

        X["Date"] = pd.to_datetime(X["Date"])
        X = X.sort_values(by="Date")

        if self.is_train and 'Weekly_Sales' in X.columns:
            X = X[X["Weekly_Sales"] > 0]
            if self.q_low is not None and self.q_high is not None:
                X = X[(X["Weekly_Sales"] >= self.q_low) & (X["Weekly_Sales"] <= self.q_high)]

        return X


class FeatureEngineer(BaseEstimator, TransformerMixin):
    def __init__(self, is_train=True):
        self.is_train = is_train

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()

        if self.is_train and 'Weekly_Sales' in X.columns:
            for lag in [1, 2, 4, 52]:
                X[f"lag_{lag}"] = X.groupby("Store")["Weekly_Sales"].shift(lag)
            X["rolling_mean_4"] = X.groupby("Store")["Weekly_Sales"].shift(1).rolling(window=4).mean()
            X["rolling_std_4"] = X.groupby("Store")["Weekly_Sales"].shift(1).rolling(window=4).std()

        markdown_cols = [col for col in X.columns if "MarkDown" in col]
        X[markdown_cols] = X[markdown_cols].fillna(0)

        X["Year"] = X["Date"].dt.year
        X["Month"] = X["Date"].dt.month
        X["Week"] = X["Date"].dt.isocalendar().week
        X["DayOfWeek"] = X["Date"].dt.dayofweek
        X["IsMonthStart"] = X["Date"].dt.is_month_start.astype(int)
        X["IsMonthEnd"] = X["Date"].dt.is_month_end.astype(int)
        X["Quarter"] = X["Date"].dt.quarter

        X.drop(columns=["Date"], inplace=True)
        return X


class MissingValueHandler(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.fill_values = {}
        pass

    def fit(self, X, y=None):
        # Store mean values for numerical columns
        numeric_cols = X.select_dtypes(include=[np.number]).columns
        for col in numeric_cols:
            if X[col].isnull().any():
                self.fill_values[col] = X[col].mean()
        return self

    def transform(self, X):
        X = X.copy()

        # Fill numerical columns with stored means
        for col, fill_value in self.fill_values.items():
            if col in X.columns:
                X[col] = X[col].fillna(fill_value)

        # Drop rows with any remaining missing values (from lag features)
        X = X.dropna()

        return X

class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.numeric_cols = []
        self.categorical_cols = []

    def fit(self, X, y=None):
        self.numeric_cols = X.select_dtypes(include=np.number).columns.tolist()
        self.categorical_cols = X.select_dtypes(include=["object", "category", "bool"]).columns.tolist()
        return self

    def transform(self, X):
        # This transformer just passes through the data but stores column info
        return X

    def get_feature_names_out(self):
        return self.numeric_cols, self.categorical_cols

In [26]:
# from sklearn.pipeline import Pipeline
# from lightgbm import LGBMRegressor

# full_pipeline = Pipeline([
#     ('merger', DataMerger(features_df, stores_df)),
#     ('cleaner', DataCleaner()),
#     ('feature_engineer', FeatureEngineer()),
#     ('missing_handler', MissingValueHandler()),
#     ('feature_selector', FeatureSelector()),
#     ('model', LGBMRegressor(
#         n_estimators=800,
#         max_depth=-1,
#         learning_rate=0.1,
#         random_state=42,
#         n_jobs=-1,
#         num_leaves=63
#     ))
# ])

# Define preprocessing after FeatureSelector
preprocessor = ColumnTransformer(
    transformers=[
        ("num", "passthrough", []),  # to be filled after fit
        ("cat", OneHotEncoder(handle_unknown="ignore"), [])
    ]
)

# Full pipeline with preprocessing + model
full_pipeline = Pipeline([
    ('merger', DataMerger(features_df, stores_df)),
    ('cleaner', DataCleaner(is_train=False)),
    ('feature_engineer', FeatureEngineer(is_train=False)),
    ('missing_handler', MissingValueHandler()),
    ('feature_selector', FeatureSelector()),  # No transform here
    ('preprocessor', preprocessor),          # You’ll configure this dynamically
    ('model', LGBMRegressor(
        n_estimators=800,
        max_depth=-1,
        learning_rate=0.1,
        random_state=42,
        n_jobs=-1,
        num_leaves=63
    ))
])


In [27]:
# ----------------------
# Training and Results
# ----------------------
def train_and_evaluate(full_pipeline, train_df, test_df, features_df, stores_df):
    """Complete training pipeline with results"""

    with mlflow.start_run(run_name="Walmart_Pipeline_Training"):

        print("🔄 Starting pipeline training...")

        # Step 1: Process all training data first
        temp_pipeline = Pipeline([
            ('merger', DataMerger(features_df, stores_df)),
            ('cleaner', DataCleaner()),
            ('feature_engineer', FeatureEngineer()),
            ('missing_handler', MissingValueHandler()),
            ('feature_selector', FeatureSelector())
        ])

        # Get processed data
        X_processed = temp_pipeline.fit_transform(train_df)

        # Configure preprocessor columns
        numeric_cols = X_processed.select_dtypes(include=[np.number]).columns.tolist()
        categorical_cols = X_processed.select_dtypes(include=["object", "category", "bool"]).columns.tolist()

        # Remove target from numeric columns if present
        if 'Weekly_Sales' in numeric_cols:
            numeric_cols.remove('Weekly_Sales')

        # Update preprocessor
        full_pipeline.named_steps['preprocessor'].transformers = [
            ("num", "passthrough", numeric_cols),
            ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols)
        ]

        print(f"📈 Numeric columns: {len(numeric_cols)}")
        print(f"📊 Categorical columns: {len(categorical_cols)}")

        # Step 2: Separate features and target from processed data
        X_features = X_processed.drop(columns=["Weekly_Sales"]) if "Weekly_Sales" in X_processed.columns else X_processed
        y = X_processed["Weekly_Sales"] if "Weekly_Sales" in X_processed.columns else None

        print(f"🔄 Total processed samples: {len(X_features)}")

        # Step 3: Train-test split on processed data
        X_train, X_test, y_train, y_test = train_test_split(
            X_features, y, test_size=0.2, random_state=42
        )

        print(f"🔄 Training set size: {len(X_train)}")
        print(f"🔄 Test set size: {len(X_test)}")

        # Step 4: Create a simplified pipeline for final preprocessing + model
        # Since we already did most preprocessing, we just need the final steps
        final_pipeline = Pipeline([
            ('preprocessor', full_pipeline.named_steps['preprocessor']),
            ('model', full_pipeline.named_steps['model'])
        ])

        # Step 5: Fit the final pipeline
        print("🚀 Training final pipeline...")
        final_pipeline.fit(X_train, y_train)

        # Step 6: Make predictions
        print("🔮 Making predictions...")
        y_pred_train = final_pipeline.predict(X_train)
        y_pred_test = final_pipeline.predict(X_test)

        # Step 7: Get IsHoliday for WMAE calculation
        is_holiday_test = X_test["IsHoliday"].astype(bool).values if "IsHoliday" in X_test.columns else np.zeros_like(y_test, dtype=bool)

        # Step 8: Calculate metrics
        train_wmae = weighted_mae(y_train.values, y_pred_train, np.zeros_like(y_train, dtype=bool))
        test_wmae = weighted_mae(y_test.values, y_pred_test, is_holiday_test)

        train_mae = mean_absolute_error(y_train, y_pred_train)
        test_mae = mean_absolute_error(y_test, y_pred_test)

        # Step 9: Print results
        print("\n" + "="*50)
        print("📊 TRAINING RESULTS")
        print("="*50)
        print(f"🎯 Training WMAE: {train_wmae:.2f}")
        print(f"🎯 Test WMAE: {test_wmae:.2f}")
        print(f"📈 Training MAE: {train_mae:.2f}")
        print(f"📈 Test MAE: {test_mae:.2f}")
        print(f"🔍 Holiday samples in test: {is_holiday_test.sum()}")
        print("="*50)

        # Step 10: Log to MLflow
        mlflow.log_params({
            "model_type": "LGBMRegressor",
            "n_estimators": 800,
            "max_depth": -1,
            "learning_rate": 0.1,
            "num_leaves": 63,
            "train_size": len(X_train),
            "test_size": len(X_test),
            "numeric_features": len(numeric_cols),
            "categorical_features": len(categorical_cols)
        })

        mlflow.log_metrics({
            "train_wmae": train_wmae,
            "test_wmae": test_wmae,
            "train_mae": train_mae,
            "test_mae": test_mae
        })

        # Step 11: Feature importance
        lgb_model = final_pipeline.named_steps['model']
        feature_names = [f"feature_{i}" for i in range(len(lgb_model.feature_importances_))]

        importance_df = pd.DataFrame({
            'feature': feature_names,
            'importance': lgb_model.feature_importances_
        }).sort_values('importance', ascending=False)

        print(f"\n🔝 Top 10 Most Important Features:")
        print("-" * 40)
        for i, (_, row) in enumerate(importance_df.head(10).iterrows()):
            print(f"{i+1:2d}. {row['feature']:<20} {row['importance']:.4f}")

        # Step 12: Create complete pipeline for saving/prediction
        complete_pipeline = Pipeline([
            ('preprocessing', temp_pipeline),
            ('final_model', final_pipeline)
        ])

        # Step 13: Save models
        joblib.dump(complete_pipeline, "walmart_complete_pipeline.pkl")
        mlflow.log_artifact("walmart_complete_pipeline.pkl")

        # mlflow.sklearn.log_model(
        #     complete_pipeline,
        #     artifact_path="complete_pipeline",
        #     registered_model_name="walmart_sales_pipeline"
        # )
        mlflow.sklearn.log_model(complete_pipeline, artifact_path="pipeline")
        joblib.dump(complete_pipeline, "walmart_sales_pipeline")

        print("✅ Training completed successfully!")
        print(f"🎯 Final Test WMAE: {test_wmae:.2f}")

        return complete_pipeline, {
            'train_wmae': train_wmae,
            'test_wmae': test_wmae,
            'train_mae': train_mae,
            'test_mae': test_mae
        }

# def make_test_predictions(pipeline, test_df):
#     """Make predictions on test data"""
#     print("🔮 Making predictions on test data...")

#     predictions = pipeline.predict(test_df)

#     submission_df = pd.DataFrame({
#         'Id': test_df['Id'] if 'Id' in test_df.columns else range(len(predictions)),
#         'Weekly_Sales': predictions
#     })

#     submission_df.to_csv("walmart_submission.csv", index=False)

#     print(f"📊 Generated {len(predictions)} predictions")
#     print(f"📊 Prediction range: {predictions.min():.2f} to {predictions.max():.2f}")
#     print("💾 Submission saved as 'walmart_submission.csv'")

#     return submission_df

# def make_test_predictions(pipeline, test_df):
#     """Make predictions on test data"""
#     print("🔮 Making predictions on test data...")

#     # Load required files for merging and historical context
#     stores_df = pd.read_csv("stores.csv")
#     features_df = pd.read_csv("features.csv")
#     train_df = pd.read_csv("train.csv")

#     # Concatenate train and test for proper lag/rolling feature computation
#     combined_df = pd.concat([train_df, test_df], sort=False)
#     combined_df.sort_values(["Store", "Dept", "Date"], inplace=True)

#     # Mark test rows
#     combined_df["is_test"] = combined_df["Weekly_Sales"].isna()

#     # Apply only preprocessing steps (must be same order as in training)
#     temp_pipeline = Pipeline([
#         ('merger', DataMerger(features_df, stores_df)),
#         ('cleaner', DataCleaner()),
#         ('feature_engineer', FeatureEngineer()),
#         ('missing_handler', MissingValueHandler()),
#         ('feature_selector', FeatureSelector())
#     ])

#     full_processed = temp_pipeline.fit_transform(combined_df)

#     # Separate test data after transformation
#     test_processed = full_processed[full_processed["is_test"] == True].drop(columns=["is_test", "Weekly_Sales"], errors='ignore')

#     # Drop Id if needed
#     ids = test_processed["Id"] if "Id" in test_processed.columns else range(len(test_processed))
#     if "Id" in test_processed.columns:
#         test_processed = test_processed.drop(columns=["Id"])

#     # Get the final model step
#     model_pipeline = pipeline.named_steps["final_model"]

#     # Make predictions
#     predictions = model_pipeline.predict(test_processed)

#     submission_df = pd.DataFrame({
#         "Id": ids,
#         "Weekly_Sales": predictions
#     })

#     submission_df.to_csv("walmart_submission.csv", index=False)

#     print(f"📊 Generated {len(predictions)} predictions")
#     print(f"📊 Prediction range: {predictions.min():.2f} to {predictions.max():.2f}")
#     print("💾 Submission saved as 'walmart_submission.csv'")

#     return submission_df


# ----------------------
# Main execution
# ----------------------
if __name__ == "__main__":
    try:
        # Load data
        train_df = pd.read_csv('train.csv')
        test_df = pd.read_csv('test.csv')
        stores_df = pd.read_csv('stores.csv')
        features_df = pd.read_csv('features.csv')

        print("🚀 Starting Walmart Sales Forecasting Pipeline")
        print("=" * 60)

        # Train the model
        pipeline, results = train_and_evaluate(full_pipeline, train_df, test_df, features_df, stores_df)

        # # Make test predictions
        # print("\n" + "="*50)
        # print("📝 MAKING TEST PREDICTIONS")
        # print("="*50)

        # submission_df = make_test_predictions(pipeline, test_df)

        # Final summary
        print("\n" + "="*60)
        print("🎉 PIPELINE EXECUTION COMPLETE!")
        print("="*60)
        print(f"✅ Best Test WMAE: {results['test_wmae']:.2f}")
        # print(f"✅ Best Test MAE: {results['test_mae']:.2f}")
        print(f"💾 Pipeline saved: walmart_complete_pipeline.pkl")
        # print(f"📝 Submission file: walmart_submission.csv")
        print("🎯 Ready for Kaggle submission!")

    except Exception as e:
        print(f"❌ Pipeline failed: {e}")
        raise

🚀 Starting Walmart Sales Forecasting Pipeline
🔄 Starting pipeline training...
📈 Numeric columns: 25
📊 Categorical columns: 2
🔄 Total processed samples: 413212
🔄 Training set size: 330569
🔄 Test set size: 82643
🚀 Training final pipeline...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.132203 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4072
[LightGBM] [Info] Number of data points in the train set: 330569, number of used features: 29
[LightGBM] [Info] Start training from score 14887.537073
🔮 Making predictions...

📊 TRAINING RESULTS
🎯 Training WMAE: 1684.50
🎯 Test WMAE: 1937.71
📈 Training MAE: 1684.50
📈 Test MAE: 1866.04
🔍 Holiday samples in test: 5791

🔝 Top 10 Most Important Features:
----------------------------------------
 1. feature_1            15147.0000
 2. feature_0            4864.0000
 3. feature_11           4581.0000
 4

2025/07/31 15:19:56 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Walmart_Pipeline_Training at: https://dagshub.com/ekvirika/WalmartRecruiting.mlflow/#/experiments/0/runs/bd5f65bbb816478183fa1df1770de227
🧪 View experiment at: https://dagshub.com/ekvirika/WalmartRecruiting.mlflow/#/experiments/0
❌ Pipeline failed: INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@dagshub.com'}


RestException: INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@dagshub.com'}